In [1]:
%load_ext autoreload
%autoreload 2
!hostname
!pwd
import os, sys
print(sys.executable)

fish-gcp005
/home/akarshkumar0101/nca-alife
/home/akarshkumar0101/miniconda3/envs/nca-alife-jax/bin/python


In [2]:
import os, sys, glob, pickle
from functools import partial

import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
import seaborn as sns
# sns.set_theme()

from tqdm.auto import tqdm
from einops import rearrange, reduce, repeat

In [4]:
# import experiment_utils
import time
import subprocess

# Launch Experiment (on this node)

In [10]:
header = '\n'.join([
    '#!/bin/bash',
    
    # 'source ~/.virtualenvs/nca-alife-torch/bin/activate'
    
    'source ~/miniconda3/etc/profile.d/conda.sh',
    'conda activate nca-alife-torch',
    
    'cd ~/nca-alife/src'
])


gpus = [4, 5, 6, 7]
log_dir = os.path.expanduser(f'~/nca-alife-data/logs/{time.strftime("%Y_%m_%d_%H_%M_%S")}')
exp_sh = f'./src/experiments/main.sh'
with open(exp_sh, 'r') as f:
    commands = [c for c in f.read().split('\n') if c]
print(f"{exp_sh} has {len(commands)} commands: ")
print('\n'.join(commands[:3]), '\n...')
print(f"Logging to {log_dir}")

./src/experiments/main.sh has 25 commands: 
python train_nca_temporal_emergence.py --seed=0 --save_dir="/home/akarshkumar0101/nca-alife-data/nca_optflow/0_0.1000_0.0000"   --dtype="float32" --device="cuda:0" --img_size=128 --d_state=16 --perception="fullconv" --kernel_size=3 --init_state="randn" --padding_mode="circular" --dt=0.01 --dropout=0.5 --prompts="a diverse ecosystem of cells moving around" --spatial_scales="1.0" --clip_model="clip-vit-base-patch32" --coef_alignment=1.0 --optical_flow_mag=0.1  --coef_optical_flow=0.0  --rollout_steps=1024 --bptt_steps=16 --pool_size=1024 --bs=8 --lr=0.0005 --n_iters=100000 --clip_grad_norm=1.0
python train_nca_temporal_emergence.py --seed=0 --save_dir="/home/akarshkumar0101/nca-alife-data/nca_optflow/0_0.1000_0.0100"   --dtype="float32" --device="cuda:0" --img_size=128 --d_state=16 --perception="fullconv" --kernel_size=3 --init_state="randn" --padding_mode="circular" --dt=0.01 --dropout=0.5 --prompts="a diverse ecosystem of cells moving around"

In [11]:
print(log_dir)

/home/akarshkumar0101/nca-alife-data/logs/2024_07_28_11_05_17


In [12]:
!mkdir -p {log_dir}
!rm {os.path.dirname(log_dir)+'/latest'}
!ln -sf {log_dir} {os.path.dirname(log_dir)+'/latest'}
for i_gpu, gpu in enumerate(gpus):
    job_ids = np.arange(len(commands))[i_gpu::len(gpus)]
    cvd = f'export CUDA_VISIBLE_DEVICES={gpu}'
    txt = f"{header}\n{cvd}\n\n"
    for job_id in job_ids:
        txt += f"touch {log_dir}/job_{job_id:05d}.status\n"
        txt += f"{commands[job_id]} &> {log_dir}/job_{job_id:05d}.out\n"
        txt += f"echo $? > {log_dir}/job_{job_id:05d}.status\n\n"
    with open(f"{log_dir}/gpu_{gpu}.sh", "w") as f:
        f.write(txt)

In [13]:
print('\n'.join(txt.split('\n')[:24]))

#!/bin/bash
source ~/miniconda3/etc/profile.d/conda.sh
conda activate nca-alife-torch
cd ~/nca-alife/src
export CUDA_VISIBLE_DEVICES=7

touch /home/akarshkumar0101/nca-alife-data/logs/2024_07_28_11_05_17/job_00003.status
python train_nca_temporal_emergence.py --seed=0 --save_dir="/home/akarshkumar0101/nca-alife-data/nca_optflow/0_0.1000_1.0000"   --dtype="float32" --device="cuda:0" --img_size=128 --d_state=16 --perception="fullconv" --kernel_size=3 --init_state="randn" --padding_mode="circular" --dt=0.01 --dropout=0.5 --prompts="a diverse ecosystem of cells moving around" --spatial_scales="1.0" --clip_model="clip-vit-base-patch32" --coef_alignment=1.0 --optical_flow_mag=0.1  --coef_optical_flow=1.0  --rollout_steps=1024 --bptt_steps=16 --pool_size=1024 --bs=8 --lr=0.0005 --n_iters=100000 --clip_grad_norm=1.0 &> /home/akarshkumar0101/nca-alife-data/logs/2024_07_28_11_05_17/job_00003.out
echo $? > /home/akarshkumar0101/nca-alife-data/logs/2024_07_28_11_05_17/job_00003.status

touch /home

In [14]:
for i_gpu, gpu in enumerate(gpus):
    command = f"bash {log_dir}/gpu_{gpu}.sh"
    print(command)
    process = subprocess.Popen(command, shell=True)

bash /home/akarshkumar0101/nca-alife-data/logs/2024_07_28_11_05_17/gpu_4.sh
bash /home/akarshkumar0101/nca-alife-data/logs/2024_07_28_11_05_17/gpu_5.sh
bash /home/akarshkumar0101/nca-alife-data/logs/2024_07_28_11_05_17/gpu_6.sh
bash /home/akarshkumar0101/nca-alife-data/logs/2024_07_28_11_05_17/gpu_7.sh


# Track Experiment

In [30]:
from collections import defaultdict

statuses = []
for job_id in range(len(commands)):
    try:
        with open(f"{log_dir}/job_{job_id:05d}.status", "r") as f:
            status = int(f.read().strip())
            statuses.append(status)
    except Exception as e:
        pass
        # print(e)
        # break
statuses = np.array(statuses)

In [31]:
(statuses==0).sum(), len(commands)

(25, 25)

In [32]:
np.unique(statuses, return_counts=True)

(array([0]), array([25]))

In [33]:
for job_id in list(range(len(commands))):
    print()
    print(f'------------------------------------------------------ {job_id} ------------------------------------------------------')
    print()
    try:
        with open(f"{log_dir}/job_{job_id:05d}.out", "r") as f:
            print(f.readlines()[-1].strip())
    except Exception as e:
        # pass
        break
        print(e)


------------------------------------------------------ 0 ------------------------------------------------------

100%|██████████| 100000/100000 [1:48:35<00:00, 15.35it/s, loss=-0.351]

------------------------------------------------------ 1 ------------------------------------------------------

100%|██████████| 100000/100000 [1:49:52<00:00, 15.17it/s, loss=-0.358]

------------------------------------------------------ 2 ------------------------------------------------------

100%|██████████| 100000/100000 [1:51:32<00:00, 14.94it/s, loss=-0.351]

------------------------------------------------------ 3 ------------------------------------------------------

100%|██████████| 100000/100000 [1:52:19<00:00, 14.84it/s, loss=-0.329]

------------------------------------------------------ 4 ------------------------------------------------------

100%|██████████| 100000/100000 [1:47:31<00:00, 15.50it/s, loss=-0.214]

------------------------------------------------------ 5 -----------------

In [34]:
log_dir

'/home/akarshkumar0101/nca-alife-data/logs/2024_07_28_11_05_17'